In [1]:
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.calibration import calibration_curve
import matplotlib.colors as mcolors
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
import numpy as np
import matplotlib
import pickle
import sys

matplotlib.use("Agg")


## get Boxplot and raincloudplot of results

In [2]:
def boxplot(column, name, name_aug):
    df = pd.read_pickle('./Results_' + name + '_' + name_aug + '.pkl')
    dat = df[column].tolist()
    print(column)
    print('std:', np.std(dat))
    print('median:', np.median(dat))
    print('mean:', np.mean(dat))
    data = [dat]
    fig = plt.figure(figsize =(10, 7))
    ax = fig.add_axes([0, 0, 1, 1])
    plt.title("Boxplot of " + column)
    bp = ax.boxplot(data, showmeans=True)
    ax.set_ylim(0.5,1.0)
    plt.savefig('boxplot_' + column + '_' + name + '_' + name_aug + '.png', bbox_inches='tight')
    plt.close()
    
    raincloudplot(column, name, name_aug)
    

In [3]:
def raincloudplot(column, name, name_aug):
    
    fig, ax = plt.subplots(figsize=(8, 2))

    boxplot_colors = ['yellowgreen']
    violin_colors = ['purple']
    scatter_colors = ['tomato']

    df = pd.read_pickle('./Results_' + name + '_' + name_aug + '.pkl')
    data = [df[column].tolist()]

    bp = ax.boxplot(data, patch_artist=True, vert=False)

    for patch, color in zip(bp['boxes'], boxplot_colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.4)

    vp = ax.violinplot(data, points=500, 
                       showmeans=False, showextrema=False, showmedians=False, vert=False)

    for idx, b in enumerate(vp['bodies']):
        m = np.mean(b.get_paths()[0].vertices[:, 0])
        b.get_paths()[0].vertices[:, 1] = np.clip(b.get_paths()[0].vertices[:, 1], idx + 1, idx + 2)
        b.set_color(violin_colors[idx])

    for idx, features in enumerate(data):
        y = np.full(len(features), idx + .8)
        idxs = np.arange(len(y))
        out = y.astype(float)
        out.flat[idxs] += np.random.uniform(low=-.1, high=.1, size=len(idxs))
        y = out
        plt.scatter(features, y, s=.3, c=scatter_colors[idx])

    plt.yticks(np.arange(1, 2, 1), [name])
    plt.xlabel('Scores')
    ax.set_xlim(0.5,1.0)
    plt.title("RainCloudPlot of " + column)
    plt.savefig('raincloudplot_' + column + '_' + name + '_' + name_aug + '.png', bbox_inches='tight')
    plt.close()


## confusion matrix

In [4]:
def confusionMatrix(name, name_aug, mapping):
    df_results = pd.read_pickle('./Results_' + name + '_' + name_aug + '.pkl')
    
    y_true = np.concatenate(df_results['classes_true'].tolist()).ravel()
    y_pred = np.concatenate(df_results['classes_x'].tolist()).ravel()

    mapping = {key: np.argmax(value) for key, value in mapping.items()}
    reversed_mapping = {value: key for key, value in mapping.items()}
    true_mapped_list = [reversed_mapping[key] for key in y_true]
    pred_mapped_list = [reversed_mapping[key] for key in y_pred]
    cm_labels = np.unique(true_mapped_list)
                  
    matrix = confusion_matrix(true_mapped_list, pred_mapped_list)
        
    df_cfm = pd.DataFrame(matrix, index=cm_labels, columns=cm_labels)
    
    plt.figure(figsize = (10,7))
    cfm_plot = sn.heatmap(df_cfm, annot=True, fmt='g')
    
    plt.xlabel('Predicted label', fontsize=12)
    plt.ylabel('True label', fontsize=12)
    cfm_plot.set_xticklabels(cfm_plot.get_xticklabels(), rotation=45, ha='right', rotation_mode='anchor')
    cfm_plot.set_yticklabels(cfm_plot.get_yticklabels(), rotation=45, ha='right', rotation_mode='anchor')
    plt.tight_layout()
    
    cfm_plot.figure.savefig("confusion_matrix.png")
    
    plt.close()
        

## time diagram for test

In [2]:
def time_diagramm():
    df_results = pd.read_pickle('./predictions.pkl')

    sr = 16000
    df_results['time_sec_mid'] = (df_results['samples_begin'] + df_results['samples_end']) / 2 / sr
    df_results['pred_0'] = df_results['predictions'].apply(lambda x: x[0])
    df_results['pred_1'] = df_results['predictions'].apply(lambda x: x[1])

    plt.figure(figsize=(8, 4))
    plt.plot(df_results['time_sec_mid'], df_results['pred_0'], marker='o', label='Prediction Std.')
    plt.plot(df_results['time_sec_mid'], df_results['pred_1'], marker='o', label='Prediction Dialect')
    plt.ylim([0, 1])
    plt.xlabel('Time (seconds)')
    plt.ylabel('Prediction value')
    plt.title('Predictions Over Time')
    plt.legend()
    plt.grid(True)
    plt.savefig("time_diagramm.png")
